In [1]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

i:\fuse\Csa\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
llm = ChatOpenAI(
    api_key = os.getenv("OPENROUTER_API_KEY"),
    base_url = "https://openrouter.ai/api/v1",
    model = "google/gemini-2.5-flash",
    max_completion_tokens=200

)

In [3]:
INTENT_PROMPT = """You are an IntentParser for a restaurant assistant.

Your job is to:
1. Classify the user's message into one or more allowed intents.
2. Extract the slots defined for each intent.

Allowed intents:
- menu — user asks about items, availability, filtering (veg/non-veg/price/type), or search.
- knowledge_base — user asks about restaurant policies, FAQs, or other general information.
- chit_chat — casual conversation or greetings.
- human_escalation — tasks that require human intervention (e.g., placing an order, complaints).
- ambiguous — cannot confidently classify.

Output format (for storing in state):
- Produce a list of tasks, where each task contains:
  - intent: one of the allowed intents
  - slots: dictionary with relevant keys (see below)
  - tool_name: string indicating which tool to call for this intent (menu_tool, kb_tool, or None)
  - result: null (to be filled later by tool)

Slot schemas:
- menu:
    - search: string | null
    - type: "veg" | "nonveg" | null
    - price_min: number | null
    - price_max: number | null
- knowledge_base:
    - query: string | null
- chit_chat:
    - slots: empty dictionary {}
- human_escalation:
    - slots: empty dictionary {}
- ambiguous:
    - slots: empty dictionary {}

Rules:
1. For menu intent, always return the same keys (`search`, `type`, `price_min`, `price_max`). Use null if missing.
2. For knowledge_base, extract the part of the user query relevant to retrieving data as `query`.
3. If multiple distinct intents are present in the user query, split them into separate tasks.
4. Include `tool_name` as `"menu_tool"` for menu, `"kb_tool"` for knowledge_base, and `None` for chit_chat, human_escalation, or ambiguous.
5. Always set `result` to null (tools will fill it later).
6. Return **only the list of tasks**; do not generate any final answer.

Few-shot examples:

User: "Do you have veg pizzas under 12?"
Tasks:
[
  {
    "intent": "menu",
    "slots": {"search":"pizza","type":"veg","price_min":null,"price_max":12},
    "tool_name": "menu_tool",
    "result": null
  }
]

User: "What time do you close on Fridays?"
Tasks:
[
  {
    "intent": "knowledge_base",
    "slots": {"query":"closing hours on Fridays"},
    "tool_name": "kb_tool",
    "result": null
  }
]

User: "Hi, how are you?"
Tasks:
[
  {
    "intent": "chit_chat",
    "slots": {},
    "tool_name": null,
    "result": null
  }
]

User: "I want to place an order for two burgers."
Tasks:
[
  {
    "intent": "human_escalation",
    "slots": {},
    "tool_name": null,
    "result": null
  }
]

User: "Tell me if you have paneer pizza and your delivery areas."
Tasks:
[
  {
    "intent": "menu",
    "slots": {"search":"pizza","type":"veg","price_min":null,"price_max":null},
    "tool_name": "menu_tool",
    "result": null
  },
  {
    "intent": "knowledge_base",
    "slots": {"query":"delivery areas"},
    "tool_name": "kb_tool",
    "result": null
  }
]

Final instruction:
Respond only with the JSON array of tasks as described above. Do not include commentary or extra text.

"""

In [4]:
from typing import TypedDict, List, Dict, Any
from langchain.schema import BaseMessage

class TaskDict(TypedDict):
    intent: str                 # "menu", "knowledge_base", "chitchat", etc.
    confidence: float           # model confidence
    slots: Dict[str, Any]       # plain dict of slots
    result: Any                 # placeholder for tool output (can be None)

class State(TypedDict):
    messages: List[BaseMessage]   # conversation history
    tasks: List[TaskDict]         # list of classified tasks

In [5]:
from pydantic import BaseModel
from typing import Literal
class Task(BaseModel):
    intent: Literal["menu", "knowledge_base", "chitchat", "human_escalation", "ambiguous"]
    confidence: float
    slots: Dict[str, Any] = {}

class IntentOutput(BaseModel):
    tasks: List[Task]

In [6]:
from langchain.schema import SystemMessage, BaseMessage
def intent_parser(state: Dict[str, Any], INTENT_PROMPT: str, llm) -> Dict[str, Any]:

    # prepare messages (SystemMessage + conversation)
    messages: List[BaseMessage] = [SystemMessage(content=INTENT_PROMPT)] + state.get("messages", [])

    # get a structured LLM that returns IntentOutput
    llm_structured = llm.with_structured_output(IntentOutput)

    # invoke model (it returns a parsed IntentOutput instance)
    resp = llm_structured.invoke(messages)  # resp is a Pydantic-like object matching IntentOutput

    normalized_tasks = []
    for t in resp.tasks:
        slots = t.slots or {}
        normalized_tasks.append({
            "intent": t.intent,
            "confidence": float(t.confidence or 0.0),
            "slots": slots,
            "result": None
        })

    # Return minimal state fragment
    return {
        "messages": [resp],      # model message
        "tasks": normalized_tasks
    }

In [7]:
q = "do you guys do anything spicy but it has to be veg and also what time do you guys open"
from langchain.schema import HumanMessage
state: State = {"messages": [HumanMessage(content=q)],
                "intent": None, "confidence": None, "slots": {}}
state = intent_parser(state,INTENT_PROMPT, llm)

In [8]:
state["tasks"]

[{'intent': 'menu',
  'confidence': 1.0,
  'slots': {'search': 'spicy',
   'type': 'veg',
   'price_min': None,
   'price_max': None},
  'result': None},
 {'intent': 'knowledge_base',
  'confidence': 1.0,
  'slots': {'query': 'opening time'},
  'result': None}]

In [9]:
state["tasks"][0]["slots"]

{'search': 'spicy', 'type': 'veg', 'price_min': None, 'price_max': None}

In [10]:
import requests

def menu_tool(base_url: str, params: dict):
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()  # raises HTTPError for bad responses (4xx/5xx)
        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"Error calling backend: {e}")
        return None


In [11]:
def kb_tool(params:dict):
    return "kb here"

In [12]:
BASE_URL = "http://127.0.0.1:8000/items"

In [13]:
for task in state["tasks"]:
    if task["intent"] == "menu":
        task["result"] = menu_tool(BASE_URL, task["slots"])
    elif task["intent"] == "knowledge_base":
        task["result"] = kb_tool(task["slots"])

In [14]:
state["tasks"]

[{'intent': 'menu',
  'confidence': 1.0,
  'slots': {'search': 'spicy',
   'type': 'veg',
   'price_min': None,
   'price_max': None},
  'result': [{'category_id': 1,
    'subcategory': 'Veg',
    'name': 'Mustang Aloo',
    'description': 'A spicy and tangy potato dish from the Mustang region of Nepal.',
    'is_available': True,
    'id': 13,
    'variations': [{'item_id': 13,
      'label': 'Default',
      'final_price': '320.00',
      'is_available': True,
      'id': 13}]},
   {'category_id': 1,
    'subcategory': 'Veg',
    'name': 'Paneer Chilly',
    'description': 'Cubes of paneer (cottage cheese) cooked in a spicy chili sauce.',
    'is_available': True,
    'id': 15,
    'variations': [{'item_id': 15,
      'label': 'Default',
      'final_price': '360.00',
      'is_available': True,
      'id': 15}]}]},
 {'intent': 'knowledge_base',
  'confidence': 1.0,
  'slots': {'query': 'opening time'},
  'result': 'kb here'}]

In [15]:
import requests

BASE_URL = "http://127.0.0.1:8000/items"
params = {
    "search": "spicy",
    "similarity_threshold": 0.2
}

res = requests.get(BASE_URL, params=state["tasks"][0]["slots"])
print(res.status_code)
print(res.json())


200
[{'category_id': 1, 'subcategory': 'Veg', 'name': 'Mustang Aloo', 'description': 'A spicy and tangy potato dish from the Mustang region of Nepal.', 'is_available': True, 'id': 13, 'variations': [{'item_id': 13, 'label': 'Default', 'final_price': '320.00', 'is_available': True, 'id': 13}]}, {'category_id': 1, 'subcategory': 'Veg', 'name': 'Paneer Chilly', 'description': 'Cubes of paneer (cottage cheese) cooked in a spicy chili sauce.', 'is_available': True, 'id': 15, 'variations': [{'item_id': 15, 'label': 'Default', 'final_price': '360.00', 'is_available': True, 'id': 15}]}]


In [20]:
ANSWERING_PROMPT = f"""
SYSTEM:
You are an assistant answering customer questions about a restaurant. You MUST ONLY use the facts in the SOURCES block. If the intent was menu or knowledge_base use its result to answer. Use all the content in result to provide concise answer. Otherwise for chitchat, you can reply like a customer service agent. For human_escalation, anwer that the information will be passed to the admin. If the sources don't contain the requested info, reply exactly: "I don't see that info in our sources — would you like me to check the menu, call the restaurant, or look it up online?" Do NOT hallucinate. Be concise (1-3 sentences). 

CHAT HISTORY:
{state["messages"]}

SOURCES:
{state["tasks"]}

"""

In [21]:
from langchain.schema import SystemMessage, BaseMessage
def answering_agent(state: Dict[str, Any], ANSWERING_PROMPT: str, llm) -> str:

    # prepare messages (SystemMessage + conversation)
    messages: List[BaseMessage] = [SystemMessage(content=ANSWERING_PROMPT)]

    # invoke model (it returns a parsed IntentOutput instance)
    resp = llm.invoke(messages)  # resp is a Pydantic-like object matching IntentOutput

    print(resp)

    return {
        "messages": [resp]     # model message
    }

In [22]:
answering_agent(state, ANSWERING_PROMPT, llm)

content='We have Mustang Aloo for 320.00 and Paneer Chilly for 360.00. kb here' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 509, 'total_tokens': 538, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'google/gemini-2.5-flash', 'system_fingerprint': None, 'id': 'gen-1761482669-AV5Bcg1bmdSFqkZ61Gmt', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--c0c00d0b-5255-4fb3-8140-3673795c29ab-0' usage_metadata={'input_tokens': 509, 'output_tokens': 29, 'total_tokens': 538, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}


{'messages': [AIMessage(content='We have Mustang Aloo for 320.00 and Paneer Chilly for 360.00. kb here', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 509, 'total_tokens': 538, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'google/gemini-2.5-flash', 'system_fingerprint': None, 'id': 'gen-1761482669-AV5Bcg1bmdSFqkZ61Gmt', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--c0c00d0b-5255-4fb3-8140-3673795c29ab-0', usage_metadata={'input_tokens': 509, 'output_tokens': 29, 'total_tokens': 538, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}

In [19]:
state["messages"]

[IntentOutput(tasks=[Task(intent='menu', confidence=1.0, slots={'search': 'spicy', 'type': 'veg', 'price_min': None, 'price_max': None}), Task(intent='knowledge_base', confidence=1.0, slots={'query': 'opening time'})])]